In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import plotly
import plotly_express as px
import plotly.graph_objs as go
import iplot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import umap
from sklearn import preprocessing

## Clean Data from Nomic

In [2]:
nELISA_data_input_path = pathlib.Path("./raw/nELISA_Data_UCA1_2023.04.11.csv")
MetaData_input_path = pathlib.Path("./raw/Metadata_UCA1_2023.04.11.csv")

nELISA_data_output_path = pathlib.Path("./clean/nELISA_Data_UCA1_2023.04.11.csv")
MetaData_output_path = pathlib.Path("./clean/Metadata_UCA1_2023.04.11.csv")

nELISA_data_all = pd.read_csv(nELISA_data_input_path)
MetaData = pd.read_csv(MetaData_input_path)

In [3]:
# Change the 'A1' cell format to 'A01' format
position = []
for i in MetaData["position"].astype(str):
    position.append(i[:1] + f"{0}" + i[1:])
MetaData["position"] = position
MetaData.head()

,plate_name,plate_barcode,position,cell_type,incubation inducer,inhibitor,inhibitor_concentration_value,inhibitor_concentration_unit,inhibitor_concentration,inducer1,inducer1_concentration_value,inducer1_concentration_unit,inducer1_concentration,inducer2,inducer2_concentration_value,inducer2_concentration_unit,inducer2_concentration
0,70117_20230210MM1_P1,430420,B06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN
1,70117_20230210MM1_P1,430420,C06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN
2,70117_20230210MM1_P1,430420,I06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN
3,70117_20230210MM1_P1,430420,J06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN
4,70117_20230210MM1_P1,430420,B07,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN


In [4]:
# Change column names
nELISA_data_all = nELISA_data_all.rename({"user_well_loc": "position"}, axis=1)
nELISA_data_all.head()
# MetaData.head()
# nELISA_data = pd.concat([MetaData,nELISA_data_all], axis=1)
# nELISA_data

MetaData["plate_position"] = (
    MetaData["plate_barcode"].astype(str) + "_" + MetaData["position"].astype(str)
)
MetaData["plate_position"]

nELISA_data_all["plate_position"] = (
    nELISA_data_all["user_plate_id"].astype(str)
    + "_"
    + nELISA_data_all["position"].astype(str)
)

# Fix plate naming
nELISA_data_all.replace(regex=[" and "], value="_", inplace=True)

['430420' '430418 and 430419']
['430420' '430418_430419']
['430420' '430418_430419']


In [6]:
# nELISA_data[nELISA_data['nelisa_sample_comments'].str.contains('empty_well')==False]

nELISA_data_all = nELISA_data_all[
    ~nELISA_data_all.nelisa_sample_comments.str.contains("empty_well", na=False)
]
nELISA_data_all = nELISA_data_all[
    ~nELISA_data_all.nelisa_sample_comments.str.contains("qc_fail", na=False)
]

In [7]:
# write to csv
nELISA_data_all.to_csv(nELISA_data_output_path, index=False)
MetaData.to_csv(MetaData_output_path, index=False)